# Toronto neighborhoods' data scraping, wrangling, and cluster analysis

#### First, we import necessary libraries for scraping, wrangling, and cluster analysis.

In [1]:
!pip3 install geocoder
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np
import geocoder
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

You should consider upgrading via the 'pip install --upgrade pip' command.


#### Next, we scrape the table from the wikipedia page providing the list of postal codes in Canada.

In [3]:
#get url using the requests library
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#parse html of the web page using html parser
soup = BeautifulSoup(source,'html.parser')

#open a csv file passing in the writing argument
csv_file = open('canada_data.csv','w')

#assign the column headers
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode','Borough','Neighborhood'])

#parse table from the web page
table  = soup.find('table')

#iterate over rows to fill the rows of the csv file
rows = table.find_all('tr')
for row in rows:
    csv_row = []
    for cell in row.find_all('td'):
        csv_row.append(cell.get_text())
    csv_writer.writerow(csv_row)
print('data loaded to csv file')

data loaded to csv file


#### Read the file into a pandas dataframe.

In [4]:
toronto_df = pd.read_csv('canada_data.csv')
toronto_df = toronto_df.replace('\n','', regex=True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


#### Then we remove rows without assigned Boroughs.

In [5]:
#replace 'Not assigned' with nan
toronto_df.replace("Not assigned", np.nan, inplace = True)

#drop rows with nan values in Borough column
toronto_df.dropna(subset=["Borough"], axis=0, inplace=True)

#reset index of the rows
toronto_df.reset_index(drop=True, inplace=True)


toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Afterwards, we replace cells with 'Not assigned' neighborhood so the neighborhood will be the same as the borough.

In [6]:
toronto_df.replace("", "Borough", inplace = True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Now we group neighborhoods with the same postal code.

In [8]:
toronto_df.groupby('PostalCode')
toronto_df['Neighborhood'] = toronto_df['Neighborhood'].str.replace('/',',')
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
toronto_df.shape

(103, 3)

#### Then we import latitude and longitude data of the postal code.

In [10]:
geo_url = "https://cocl.us/Geospatial_data"

geocode_df = pd.read_csv(geo_url)
geocode_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geocode_df.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


####  We merge the data in order to get a dataframe with all the columns we need.

In [11]:
toronto_df = pd.merge(toronto_df, geocode_df, on='PostalCode')
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### We then import the client ID and client secret of the Foursquare API to be able to make calls to the API.

In [12]:
CLIENT_ID = 'DBMV3YHWPSD3WL5D02DBTV3U0ANAXUUIVMAILOWW0LFC5RZO' # your Foursquare ID
CLIENT_SECRET = 'RAJ1K0YBRP3QTBRGMXURXXS5RGJLHC3PDY32XZFSHALQ3U1K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DBMV3YHWPSD3WL5D02DBTV3U0ANAXUUIVMAILOWW0LFC5RZO
CLIENT_SECRET:RAJ1K0YBRP3QTBRGMXURXXS5RGJLHC3PDY32XZFSHALQ3U1K


#### Assigning a list with the latitude and longitude values of the city of Toronto.

In [13]:
toronto_latlong = [43.653225, -79.383186]

#### getting Boroughs that only have the word "Toronto" in it

In [14]:
toronto_east = toronto_df[toronto_df['Borough'] == 'East Toronto'].reset_index(drop=True)
toronto_west = toronto_df[toronto_df['Borough'] == 'West Toronto'].reset_index(drop=True)
toronto_DT = toronto_df[toronto_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
frames = [toronto_east, toronto_west, toronto_DT]
toronto_boroughs = pd.concat(frames).set_index('PostalCode').reset_index()
toronto_boroughs.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558


#### Creat a map with markers of the boroughs in the toronto_boroughs dataframe.

In [15]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=toronto_latlong, zoom_start=11)

for lat, lng, label in zip(toronto_boroughs['Latitude'], toronto_boroughs['Longitude'], toronto_boroughs['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='##3186cc',
        fill=True,
        fill_color='##3186cc',
        fill_opacity=0.75,
        parse_html=False).add_to(toronto_map)
toronto_map

#### Let's take a certain neighborhood and explore the top 50 venues within 500 meter radius.

In [16]:
toronto_boroughs.loc[6, 'Neighborhood']

'Little Portugal , Trinity'

In [17]:
hood_lat = toronto_boroughs.loc[6, 'Latitude'] # neighborhood latitude 
hood_lon = toronto_boroughs.loc[6, 'Longitude'] # neighborhood longitude

hood_name = toronto_boroughs.loc[6, 'Neighborhood'] # neighborhood name

print('Lat and long of {} are {}, {}.'.format(hood_name, 
                                                               hood_lat, 
                                                               hood_lon))

Lat and long of Little Portugal , Trinity are 43.647926700000006, -79.4197497.


In [18]:
radius = 500
LIMIT = 50
url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, hood_lat, hood_lon, VERSION, radius, LIMIT)
url1

'https://api.foursquare.com/v2/venues/explore?client_id=DBMV3YHWPSD3WL5D02DBTV3U0ANAXUUIVMAILOWW0LFC5RZO&client_secret=RAJ1K0YBRP3QTBRGMXURXXS5RGJLHC3PDY32XZFSHALQ3U1K&ll=43.647926700000006,-79.4197497&v=20180605&radius=500&limit=50'

In [19]:
results = requests.get(url1).json()

#function that extracts venue category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Let's put the venue data in a pandas dataframe called nearby_venues.

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-20-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Bellwoods Brewery,Brewery,43.647097,-79.419955
1,Pizzeria Libretto,Pizza Place,43.648979,-79.420604
2,OddSeoul,Korean Restaurant,43.646192,-79.419601
3,Bang Bang Ice Cream & Bakery,Ice Cream Shop,43.646246,-79.419553
4,Foxley Bistro,Asian Restaurant,43.648643,-79.420495


#### Let's see how many venues are in our dataframe.

In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

41 venues were returned by Foursquare.


#### Now we define a function that gets the nearby venues for every neigborhood.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now let's run the function on the toronto_boroughs dataframe so we get the nearby venues for all neighborhoods and put it in a dataframe called toronto_venues.

In [23]:
toronto_venues = getNearbyVenues(names=toronto_boroughs['Neighborhood'],
                                   latitudes=toronto_boroughs['Latitude'],
                                   longitudes=toronto_boroughs['Longitude']
                                  )

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Business reply mail Processing CentrE
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , The Junction South
Parkdale , Roncesvalles
Runnymede , Swansea
Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosedale
Stn A PO Boxes
St. James Town , Cabbagetown
First Canadian Place , Underground city
Church and Wellesley


In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(1079, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West , Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [25]:
#group explored venues within the dataframe by neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
Business reply mail Processing CentrE,18,18,18,18,18,18
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",18,18,18,18,18,18
Central Bay Street,50,50,50,50,50,50
Christie,18,18,18,18,18,18
Church and Wellesley,50,50,50,50,50,50
"Commerce Court , Victoria Hotel",50,50,50,50,50,50
"Dufferin , Dovercourt Village",16,16,16,16,16,16


#### Now let's see how many unique types of venues are there in our dataframe

In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 207 uniques categories.


### Now we do the data preprocessing for cluster analysis.

In [27]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.00000,0.00000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.00000,0.000000
2,Business reply mail Processing CentrE,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.00000,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.00000,0.000000
4,Central Bay Street,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.00000,0.00000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.00000,0.000000
6,Church and Wellesley,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.02,0.000000,0.00,0.000000,0.000000,0.00000,0.00000,0.000000
7,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.02000,0.000000
8,"Dufferin , Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.00000,0.062500
9,"First Canadian Place , Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.00000,0.02000,0.000000


In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Seafood Restaurant,Beer Bar,Restaurant,Italian Restaurant,Farmers Market,Café,Bakery
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bar,Stadium,Burrito Place,Intersection,Italian Restaurant,Pet Store,Performing Arts Venue
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Auto Workshop,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Burrito Place
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Coffee Shop,Boat or Ferry,Rental Car Location,Bar,Plane,Harbor / Marina
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Ice Cream Shop,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Hotel,Modern European Restaurant


In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 0, 1, 4, 0, 0, 4, 4, 4], dtype=int32)

In [32]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_boroughs
toronto_merged.rename({'Neighbourhood':'Neighborhood'}, axis=1, inplace=True)

toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Health Food Store,Pub,Wine Shop,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Trail,Liquor Store,Bubble Tea Shop,Pub
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,0,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop,Park,Pet Store,Pizza Place,Burrito Place,Liquor Store,Pub,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Café,Coffee Shop,Gastropub,American Restaurant,Brewery,Bakery,Yoga Studio,Bar,Seafood Restaurant,Ice Cream Shop
4,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558,0,Light Rail Station,Yoga Studio,Auto Workshop,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Burrito Place


In [33]:
# create map
map_clusters = folium.Map(location=toronto_latlong, zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=False)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Finally we explore each cluster in a seperate dataframe and notice the similarities.

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Trail,Liquor Store,Bubble Tea Shop,Pub
2,East Toronto,0,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop,Park,Pet Store,Pizza Place,Burrito Place,Liquor Store,Pub,Brewery
4,East Toronto,0,Light Rail Station,Yoga Studio,Auto Workshop,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Burrito Place
6,West Toronto,0,Bar,Restaurant,Men's Store,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Café,Asian Restaurant,Yoga Studio,Record Shop,Brewery
9,West Toronto,0,Gift Shop,Eastern European Restaurant,Dessert Shop,Bookstore,Coffee Shop,Dog Run,Italian Restaurant,Movie Theater,Breakfast Spot,Bar
10,West Toronto,0,Coffee Shop,Café,Pub,Pizza Place,Italian Restaurant,Sushi Restaurant,Boutique,Bar,Indie Movie Theater,Bookstore
15,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Cheese Shop,Seafood Restaurant,Beer Bar,Restaurant,Italian Restaurant,Farmers Market,Café,Bakery
17,Downtown Toronto,0,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Italian Restaurant,Diner,Nightclub,Candy Store,Restaurant
26,Downtown Toronto,0,Italian Restaurant,Café,Beer Bar,Seafood Restaurant,Creperie,Cocktail Bar,Coffee Shop,Cheese Shop,Farmers Market,Restaurant
27,Downtown Toronto,0,Restaurant,Coffee Shop,Pizza Place,Bakery,Café,Italian Restaurant,Pub,Sandwich Place,Pharmacy,Beer Store


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Coffee Shop,Boat or Ferry,Rental Car Location,Bar,Plane,Harbor / Marina


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,2,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop,Bookstore,Yoga Studio,Cosmetics Shop,Pub,Brewery
2,East Toronto,2,Sandwich Place,Fast Food Restaurant,Board Shop,Brewery,Burrito Place,Pet Store,Gym,Steakhouse,Sushi Restaurant,Park
3,East Toronto,2,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Wine Bar,Fish Market,Pet Store,Park
4,East Toronto,2,Yoga Studio,Auto Workshop,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park,Butcher,Pizza Place
5,West Toronto,2,Bakery,Pharmacy,Music Venue,Grocery Store,Brewery,Bar,Bank,Café,Pizza Place,Supermarket
6,West Toronto,2,Bar,Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Café,Men's Store,Wine Bar,Korean Restaurant,Japanese Restaurant
7,West Toronto,2,Café,Breakfast Spot,Coffee Shop,Yoga Studio,Performing Arts Venue,Bar,Restaurant,Burrito Place,Pet Store,Nightclub
8,West Toronto,2,Mexican Restaurant,Café,Thai Restaurant,Furniture / Home Store,Flea Market,Park,Cajun / Creole Restaurant,Diner,Bookstore,Italian Restaurant
9,West Toronto,2,Gift Shop,Coffee Shop,Dog Run,Italian Restaurant,Eastern European Restaurant,Bookstore,Bar,Dessert Shop,Restaurant,Breakfast Spot
10,West Toronto,2,Café,Pizza Place,Coffee Shop,Pub,Sushi Restaurant,Italian Restaurant,Indie Movie Theater,Dessert Shop,Restaurant,Diner


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Trail,Health Food Store,Pub,Wine Shop,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,4,Café,Coffee Shop,Gastropub,American Restaurant,Brewery,Bakery,Yoga Studio,Bar,Seafood Restaurant,Ice Cream Shop
5,West Toronto,4,Pharmacy,Bakery,Wine Shop,Gym / Fitness Center,Park,Middle Eastern Restaurant,Pizza Place,Café,Brewery,Bar
7,West Toronto,4,Café,Breakfast Spot,Coffee Shop,Bar,Stadium,Burrito Place,Intersection,Italian Restaurant,Pet Store,Performing Arts Venue
8,West Toronto,4,Mexican Restaurant,Café,Thai Restaurant,Gastropub,Flea Market,Park,Cajun / Creole Restaurant,Diner,Bookstore,Italian Restaurant
11,Downtown Toronto,4,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Restaurant,Café,Theater,Farmers Market,Hotel
12,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Burger Joint,Bar,Hobby Shop,Beer Bar,Italian Restaurant,Japanese Restaurant
13,Downtown Toronto,4,Café,Coffee Shop,Middle Eastern Restaurant,Bookstore,Clothing Store,Theater,Ramen Restaurant,Tea Room,Restaurant,College Rec Center
14,Downtown Toronto,4,Café,Coffee Shop,Gastropub,Hotel,Farmers Market,Creperie,Seafood Restaurant,Cosmetics Shop,Gym,Optical Shop
16,Downtown Toronto,4,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Ice Cream Shop,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Hotel,Modern European Restaurant
18,Downtown Toronto,4,Coffee Shop,Café,American Restaurant,Pizza Place,Restaurant,Steakhouse,Seafood Restaurant,Concert Hall,Brazilian Restaurant,Gym / Fitness Center
